# Predicting T-shirt size using the ANSUR II dataset
We will here try to predict a persons t-shirt size given the weight and height of the person. We will use the ANSUR II dataset which contains a lot of information about the physical attributes of a large number of people. 

We will first try to map the persons in the dataset to a t-shirt size. It is hard to find a concise size chart for t-shirt so we will create our own, initial chart, based on these assumptions:

We will only look at two measurements, Shoulder Width and Chest Circumference.

Our first problem is that Shoulder Width is not one of the measurements taken in the dataset. But we have Biacromial Breadth which is the distance between the two acromion processes. We will assume that this is the same as Shoulder Width.

We will then have these initial rules:

| Size | Percentile |
|------|------------|
| XS   | 0-5        |
| S    | 5-25       |
| M    | 25-50      |
| L    | 50-75      |
| XL   | 75-90      |
| XXL  | 90-97      |
| XXXL | 97-100     |

## Inspect the data

In [1]:
import pandas as pd

female = pd.read_csv('./data/female.csv')
male = pd.read_csv('./data/male.csv')


C:\Users\AladdinElkhatib\AppData\Local\Temp\ipykernel_43824\1609720030.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
print(f'For women we have(rows, columns) {female.shape}')
print(f'For men we have(rows, columns) {male.shape}')


For women we have(rows, columns) (1986, 108)
For men we have(rows, columns) (4082, 108)


## Checking the percentiles
Let us determine the percentiles of the data.

In [3]:
def compute_percentile_ranges(column):
    #Define percentile ranges
    ranges = [(0,5), (5,25), (25,50), (50, 75), (75, 90), (90, 97), (97, 100)]

    percentiles = {(low, high): (column.quantile(low/100), column.quantile(high/100)) for low, high in ranges}

    counts = {}

    for r, (low, high) in percentiles.items():
        counts[r] = ((column >= low) & (column < high)).sum()
    
    return counts

print(compute_percentile_ranges(female['chestcircumference']))
print(compute_percentile_ranges(female['biacromialbreadth']))

print(compute_percentile_ranges(male['chestcircumference']))
print(compute_percentile_ranges(male['biacromialbreadth']))


{(0, 5): 100, (5, 25): 396, (25, 50): 492, (50, 75): 499, (75, 90): 299, (90, 97): 140, (97, 100): 59}
{(0, 5): 93, (5, 25): 377, (25, 50): 477, (50, 75): 541, (75, 90): 297, (90, 97): 139, (97, 100): 61}
{(0, 5): 199, (5, 25): 810, (25, 50): 1025, (50, 75): 1012, (75, 90): 616, (90, 97): 295, (97, 100): 124}
{(0, 5): 191, (5, 25): 787, (25, 50): 989, (50, 75): 1079, (75, 90): 610, (90, 97): 303, (97, 100): 122}


## Generate the t-shirt size chart

In [4]:
def compute_size_percentile_measurements(data, chest_column, shoulder_column):
    sizes = ['XS', 'S', 'M', 'L', 'XL', '2XL', '3XL']
    ranges = [0, 5, 25, 50, 75, 90, 97]
    
    # Compute the values for each percentile for chest and shoulder
    chest_percentiles = {p: data[chest_column].quantile(p/100) for p in ranges}
    shoulder_percentiles = {p: data[shoulder_column].quantile(p/100) for p in ranges}
   
    # Map the t-shirt sizes to the corresponding chest and shoulder measurments
    size_mappings = {}
    for i, size in enumerate(sizes):
        size_mappings[size] = {
            'Chest' : int(chest_percentiles[ranges[i]]),
            'Shoulder' : int(shoulder_percentiles[ranges[i]])
        }
        

    return size_mappings






print(compute_size_percentile_measurements(female, 'chestcircumference', 'biacromialbreadth'))
print(compute_size_percentile_measurements(male, 'chestcircumference', 'biacromialbreadth'))



{'XS': {'Chest': 695, 'Shoulder': 283}, 'S': {'Chest': 824, 'Shoulder': 335}, 'M': {'Chest': 889, 'Shoulder': 353}, 'L': {'Chest': 940, 'Shoulder': 365}, 'XL': {'Chest': 999, 'Shoulder': 378}, '2XL': {'Chest': 1057, 'Shoulder': 389}, '3XL': {'Chest': 1117, 'Shoulder': 400}}
{'XS': {'Chest': 774, 'Shoulder': 337}, 'S': {'Chest': 922, 'Shoulder': 384}, 'M': {'Chest': 996, 'Shoulder': 403}, 'L': {'Chest': 1056, 'Shoulder': 415}, 'XL': {'Chest': 1117, 'Shoulder': 428}, '2XL': {'Chest': 1172, 'Shoulder': 441}, '3XL': {'Chest': 1233, 'Shoulder': 452}}


In [5]:
female_sizes = {
    'XS': {'Chest': 695, 'Shoulder': 283}, 
    'S': {'Chest': 824, 'Shoulder': 335}, 
    'M': {'Chest': 889, 'Shoulder': 353}, 
    'L': {'Chest': 940, 'Shoulder': 365}, 
    'XL': {'Chest': 999, 'Shoulder': 378}, 
    '2XL': {'Chest': 1057, 'Shoulder': 389}, 
    '3XL': {'Chest': 1117, 'Shoulder': 400}
    }

male_sizes = {
    'XS': {'Chest': 774, 'Shoulder': 337}, 
    'S': {'Chest': 922, 'Shoulder': 384}, 
    'M': {'Chest': 996, 'Shoulder': 403}, 
    'L': {'Chest': 1056, 'Shoulder': 415}, 
    'XL': {'Chest': 1117, 'Shoulder': 428}, 
    '2XL': {'Chest': 1172, 'Shoulder': 441}, 
    '3XL': {'Chest': 1233, 'Shoulder': 452}
}


In [6]:
def get_size(data, size_chart):
    matches = {size: 0 for size in size_chart.keys()}
    ties = 0

    for _, row in data.iterrows():
        possible_sizes = []

        for size, measurements in size_chart.items():
            if (row['biacromialbreadth'] <= measurements['Shoulder'] and  
                row['chestcircumference'] <= measurements['Chest']):
                possible_sizes.append(size)

        if len(possible_sizes) == 1:
            matches[possible_sizes[0]] += 1
        elif len(possible_sizes) > 1:
            ties += 1
    
    return matches, ties


In [7]:
female_matches, female_ties = get_size(female, female_sizes)
male_matches, male_ties = get_size(male, male_sizes)

print('Female matches:', female_matches)
print('Female ties:', female_ties)
print('Male matches:', male_matches)
print('Male ties:', male_ties)



Female matches: {'XS': 0, 'S': 0, 'M': 0, 'L': 0, 'XL': 0, '2XL': 0, '3XL': 236}
Female ties: 1642
Male matches: {'XS': 0, 'S': 0, 'M': 0, 'L': 0, 'XL': 0, '2XL': 0, '3XL': 434}
Male ties: 3437


This is not good. Let us have overlapping measurments.

In [8]:
def create_overlapping_size_chart(original_chart):
    # Skapa en tom dict. för den nya storlekstabellen med överlappande mått
    overlapping_chart = {}

    #  Hämta en lista med storlekarna från den ursprungliga storlekstabellen
    sizes = list(original_chart.keys())

    # Loopa igenom varje storlek
    for i, size in enumerate(sizes):
        # Skapa en inre dict för varje storlek i den nya tabellen
        overlapping_chart[size] = {}
        
        # Hantera specialfall för första och sista storleken
        if i == 0:
            overlapping_chart[size]['Chest'] = [original_chart[size]['Chest'], original_chart[sizes[i+1]]['Chest']+5]
            overlapping_chart[size]['Shoulder'] = [original_chart[size]['Shoulder'], original_chart[sizes[i+1]]['Shoulder']+5]
        elif i == len(sizes)-1:
            overlapping_chart[size]['Chest'] = [original_chart[size]['Chest']-5, original_chart[size]['Chest']+1000]
            overlapping_chart[size]['Shoulder'] = [original_chart[size]['Shoulder']-5, original_chart[size]['Shoulder']+1000]
        else:
            overlapping_chart[size]['Chest'] = [original_chart[size]['Chest']-5, original_chart[sizes[i+1]]['Chest']+5]
            overlapping_chart[size]['Shoulder'] = [original_chart[size]['Shoulder']-5, original_chart[sizes[i+1]]['Shoulder']+5]

    return overlapping_chart




In [9]:
new_female_sizes = create_overlapping_size_chart(female_sizes)
new_male_sizes = create_overlapping_size_chart(male_sizes)

# loopar igenom varje storlek (k) och dess överlappande mått (v) för kvinnor/män och skriver ut dem

for k, v in new_female_sizes.items():
    print(k,v)
print()
for k, v in new_male_sizes.items():
    print(f"'{k}' : {v},")


XS {'Chest': [695, 829], 'Shoulder': [283, 340]}
S {'Chest': [819, 894], 'Shoulder': [330, 358]}
M {'Chest': [884, 945], 'Shoulder': [348, 370]}
L {'Chest': [935, 1004], 'Shoulder': [360, 383]}
XL {'Chest': [994, 1062], 'Shoulder': [373, 394]}
2XL {'Chest': [1052, 1122], 'Shoulder': [384, 405]}
3XL {'Chest': [1112, 2117], 'Shoulder': [395, 1400]}

'XS' : {'Chest': [774, 927], 'Shoulder': [337, 389]},
'S' : {'Chest': [917, 1001], 'Shoulder': [379, 408]},
'M' : {'Chest': [991, 1061], 'Shoulder': [398, 420]},
'L' : {'Chest': [1051, 1122], 'Shoulder': [410, 433]},
'XL' : {'Chest': [1112, 1177], 'Shoulder': [423, 446]},
'2XL' : {'Chest': [1167, 1238], 'Shoulder': [436, 457]},
'3XL' : {'Chest': [1228, 2233], 'Shoulder': [447, 1452]},


In [10]:
female_sizes = {
'XS' : {'Chest': [695, 829], 'Shoulder': [283, 340]},
'S' :{'Chest': [819, 894], 'Shoulder': [330, 358]},
'M' :{'Chest': [884, 945], 'Shoulder': [348, 370]},
'L' :{'Chest': [935, 1004], 'Shoulder': [360, 383]},
'XL' :{'Chest': [994, 1062], 'Shoulder': [373, 394]},
'2XL' :{'Chest': [1052, 1122], 'Shoulder': [384, 405]},
'3XL' :{'Chest': [1112, 2117], 'Shoulder': [395, 1400]}
}

male_sizes = {
    'XS' : {'Chest': [774, 927], 'Shoulder': [337, 389]},
    'S' : {'Chest': [917, 1001], 'Shoulder': [379, 408]},
    'M' : {'Chest': [991, 1061], 'Shoulder': [398, 420]},
    'L' : {'Chest': [1051, 1122], 'Shoulder': [410, 433]},
    'XL' : {'Chest': [1112, 1177], 'Shoulder': [423, 446]},
    '2XL' : {'Chest': [1167, 1238], 'Shoulder': [436, 457]},
    '3XL' : {'Chest': [1228, 2233], 'Shoulder': [447, 1452]},
}

In [14]:
def get_size(data, size_chart):
    # Skapa en dict för att räkna antalet matchningar för varje storlek
    matches = {size: 0 for size in size_chart.keys()}
    # Variabel för att räkna antalet situationer fär det finns flera möjliga storlekar
    ties = 0

    # Hämta en lista med storlekarna i ordning
    size_ordered = list(size_chart.keys())

    # loopa igenom varje rad i data
    for _, row in data.iterrows():
        # Lista för att hålla de möjliga storlekarna för den aktuella personen
        possible_sizes = []

        # Loopa igenom varje storlek i storlekstabellen
        for size, measurements in size_chart.items():
            # Kontrollerade om personernas mått ligger inom intervaller för den aktuella storleken
            if (row['biacromialbreadth'] >= measurements['Shoulder'][0] and 
                row['biacromialbreadth'] <= measurements['Shoulder'][1] and
                row['chestcircumference'] >= measurements['Chest'][0] and
                row['chestcircumference'] <= measurements['Chest'][1]):
                possible_sizes.append(size)

        # Om det bara finns en möjlig storlek, öka räknaren för den storleken
        if len(possible_sizes) == 1:
            matches[possible_sizes[0]] += 1
        # Om det finns flera möjliga storlekar
        elif len(possible_sizes) > 1:
            # Check if sizes are adjecent
            are_adjecent = all([abs(size_ordered.index(possible_sizes[i]) - size_ordered.index(possible_sizes[i+1])) ==1 
                                for i in range(len(possible_sizes)-1)])
            if are_adjecent:
                #Assign the larger size
                larger_size = max(possible_sizes, key = lambda s: size_ordered.index(s))
                matches[larger_size] += 1
            else:
                # Öka räknaren för situationer där det finns flera möjliga storlekar
                ties += 1
    return matches, ties 

In [12]:
result = all([True, True, False])
print(result)

x = 10
result = all([x<15, x==10, x < 12])
print(result)

In [16]:
female_matches, female_ties = get_size(female, female_sizes)
male_matches, male_ties = get_size(male, male_sizes)

print('Female matches:', female_matches)
print('Female ties:', female_ties)
print('Male matches:', male_matches)
print('Male ties:', male_ties)



Female matches: {'XS': 23, 'S': 185, 'M': 247, 'L': 276, 'XL': 118, '2XL': 35, '3XL': 13}
Female ties: 0
Male matches: {'XS': 63, 'S': 428, 'M': 578, 'L': 593, 'XL': 331, '2XL': 101, '3XL': 50}
Male ties: 0
